In [1]:
// loading movies.csv and ratings.csv as DataFrame

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

val spark = SparkSession.builder
  .appName("Movie Genre Aggregation")
  .getOrCreate()

// Load the movies and ratings CSV files into DataFrames
val moviesDF = spark.read.option("header", "true").csv("gs://first-job-bucket/movies.csv")
val ratingsDF = spark.read.option("header", "true").csv("gs://first-job-bucket/ratings.csv")

moviesDF.printSchema()
moviesDF.show(5)
ratingsDF.printSchema()
ratingsDF.show(5)

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     17|   4.0|944249077|
|     1|     25|   1.0|944250228|
|     1|     29|   2.0|943230976|
|     1|     30|   5.

spark = org.apache.spark.sql.SparkSession@184d0e8b
moviesDF = [movieId: string, title: string ... 1 more field]
ratingsDF = [userId: string, movieId: string ... 2 more fields]


[userId: string, movieId: string ... 2 more fields]

In [2]:
// Explode the genres column to create a row per genre

val explodedMoviesDF = moviesDF.withColumn("genres", explode(split(col("genres"), "\\|")))

explodedMoviesDF.show(5)

+-------+----------------+---------+
|movieId|           title|   genres|
+-------+----------------+---------+
|      1|Toy Story (1995)|Adventure|
|      1|Toy Story (1995)|Animation|
|      1|Toy Story (1995)| Children|
|      1|Toy Story (1995)|   Comedy|
|      1|Toy Story (1995)|  Fantasy|
+-------+----------------+---------+
only showing top 5 rows



explodedMoviesDF = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [3]:
// getting list of all the genres

val genresList = explodedMoviesDF.select("genres")
  .distinct()
  .collect()

val genreNames = genresList.map(_.getString(0)).toList

println(genreNames.mkString(", "))

Mystery, Musical,  We're Comin' To Get Ya!"" (2014)", Action, Romance, Thriller, Fantasy, Animation, Film-Noir, IMAX, Sci-Fi, Drama, Documentary, (no genres listed), Western, Comedy, Crime, War, Children, Adventure, Horror


genresList = Array([Mystery], [Musical], [ We're Comin' To Get Ya!"" (2014)"], [Action], [Romance], [Thriller], [Fantasy], [Animation], [Film-Noir], [IMAX], [Sci-Fi], [Drama], [Documentary], [(no genres listed)], [Western], [Comedy], [Crime], [War], [Children], [Adventure], [Horror])
genreNames = List(Mystery, Musical, " We're Comin' To Get Ya!"" (2014)"", Action, Romance, Thriller, Fantasy, Animation, Film-Noir, IMAX, Sci-Fi, Drama, Documentary, (no genres listed), Western, Comedy, Crime, War, Children, Adventure, Horror)


List(Mystery, Musical, " We're Comin' To Get Ya!"" (2014)"", Action, Romance, Thriller, Fantasy, Animation, Film-Noir, IMAX, Sci-Fi, Drama, Documentary, (no genres listed), Western, Comedy, Crime, War, Children, Adventure, Horror)

In [18]:
// RRD custom transformations to handle inconsistent genre names

import org.apache.spark.sql.Row

val moviesRdd = explodedMoviesDF.rdd

val cleanGenre: String => String = (genre: String) => {
  val cleanedGenre = genre.trim
    .replaceAll("[\\[\\]()\"']", "")
    .replaceAll("(no genres listed)", "Out Of World")

  cleanedGenre match {
    case "Sci-Fi" => "Science Fiction"
    case "Film-Noir" => "Film Noir"
    case "IMAX" => "IMAX Movie"
    case _ => cleanedGenre
  }
}

val cleanedMoviesRdd = moviesRdd.map(row => {
  val movieId = row.getAs[String]("movieId")
  val title = row.getAs[String]("title")
  val genres = row.getAs[String]("genres")

  val cleanedGenres = genres.split("\\|").map(cleanGenre).mkString("|")

  Row(movieId, title, cleanedGenres)
})

cleanedMoviesRdd.take(5).foreach(println)

[1,Toy Story (1995),Adventure]
[1,Toy Story (1995),Animation]
[1,Toy Story (1995),Children]
[1,Toy Story (1995),Comedy]
[1,Toy Story (1995),Fantasy]


moviesRdd = MapPartitionsRDD[44] at rdd at <console>:30
cleanGenre = > String = $Lambda$6272/0x00000008022e7840@30298f29
cleanedMoviesRdd = MapPartitionsRDD[109] at map at <console>:51


MapPartitionsRDD[109] at map at <console>:51

In [19]:
// Perform the join between movies and ratings on movieId using a DataFrame

val movieSchema = moviesDF.schema
val transformedMovieDF = spark.createDataFrame(cleanedMoviesRdd, movieSchema)

val joinedDF = ratingsDF.join(transformedMovieDF, "movieId")

joinedDF.show(5)

+-------+------+------+----------+-----------------+--------+
|movieId|userId|rating| timestamp|            title|  genres|
+-------+------+------+----------+-----------------+--------+
| 100040|130583|   3.0|1358952273|True Blue (2001) |   Crime|
| 100040|130583|   3.0|1358952273|True Blue (2001) |   Drama|
| 100040|130583|   3.0|1358952273|True Blue (2001) |Thriller|
| 100040|172165|   2.5|1368843341|True Blue (2001) |   Crime|
| 100040|172165|   2.5|1368843341|True Blue (2001) |   Drama|
+-------+------+------+----------+-----------------+--------+
only showing top 5 rows



movieSchema = StructType(StructField(movieId,StringType,true),StructField(title,StringType,true),StructField(genres,StringType,true))
transformedMovieDF = [movieId: string, title: string ... 1 more field]
joinedDF = [movieId: string, userId: string ... 4 more fields]


[movieId: string, userId: string ... 4 more fields]

In [22]:
// Calculating average rating for each genre

import org.apache.spark.sql.Row

val joinedRdd = joinedDF.rdd

val genreRatingsRdd = joinedRdd.flatMap(row => {
  val movieId = row.getAs[String]("movieId")
  val title = row.getAs[String]("title")
  val genres = row.getAs[String]("genres")
  
  val ratingString = row.getAs[String]("rating")
  val rating = try {
    ratingString.toDouble
  } catch {
    case e: Exception => 0.0
  }

  genres.split("\\|").map(genre => {
    (genre, (rating, 1))
  })
})

// Use reduceByKey to aggregate ratings and counts per genre
val aggregatedGenresRdd = genreRatingsRdd.reduceByKey((a, b) => {
  val (ratingSumA, countA) = a
  val (ratingSumB, countB) = b
  (ratingSumA + ratingSumB, countA + countB)
})

// Calculate the average rating for each genre
val averageRatingsRdd = aggregatedGenresRdd.mapValues {
  case (ratingSum, count) => ratingSum / count.toDouble
}

averageRatingsRdd.collect().foreach {
  case (genre, avgRating) => println(s"Genre: $genre, Average Rating: $avgRating")
}

Genre: Adventure, Average Rating: 3.5234385724723545
Genre: Action, Average Rating: 3.476407141777424
Genre: Documentary, Average Rating: 3.6911815290871948
Genre: Drama, Average Rating: 3.6824540581800784
Genre: War, Average Rating: 3.7916994435766664
Genre: Horror, Average Rating: 3.3071549944529486
Genre: Comedy, Average Rating: 3.4323858239436777
Genre: Romance, Average Rating: 3.5450028644529983
Genre: Fantasy, Average Rating: 3.512174705402107
Genre: Mystery, Average Rating: 3.673102967818112
Genre: Science Fiction, Average Rating: 3.4916991949223912
Genre: IMAX Movie, Average Rating: 3.593312447839248
Genre: Were Comin To Get Ya! 2014, Average Rating: 2.0
Genre: Thriller, Average Rating: 3.5317020152396505
Genre: Western, Average Rating: 3.6001753109842554
Genre: Film Noir, Average Rating: 3.915774014636868
Genre: Musical, Average Rating: 3.554276956937205
Genre: Children, Average Rating: 3.4392409733948646
Genre: Animation, Average Rating: 3.6153322869262636
Genre: Crime, Avera

joinedRdd = MapPartitionsRDD[138] at rdd at <console>:35
genreRatingsRdd = MapPartitionsRDD[142] at flatMap at <console>:37
aggregatedGenresRdd = ShuffledRDD[143] at reduceByKey at <console>:55
averageRatingsRdd = MapPartitionsRDD[144] at mapValues at <console>:62


MapPartitionsRDD[144] at mapValues at <console>:62

In [26]:
// Converting the RDD back to a DataFrame and save the aggregated results in Parquet format in HDFS

import org.apache.spark.sql.{Row, SparkSession}
import org.apache.spark.sql.functions._

case class GenreAverage(genre: String, average_rating: Double)

// Convert the RDD to a DataFrame
val averageRatingsDF = averageRatingsRdd.map {
  case (genre, avgRating) => GenreAverage(genre, avgRating)
}.toDF()

averageRatingsDF.show(5)

averageRatingsDF.write.parquet("hdfs://cluster-fcbe-m/spark/Day16_17/average_ratings_cs1.parquet")

+-----------+------------------+
|      genre|    average_rating|
+-----------+------------------+
|  Adventure|3.5234385724723545|
|     Action| 3.476407141777424|
|Documentary|3.6911815290871948|
|      Drama|3.6824540581800784|
|        War|3.7916994435766664|
+-----------+------------------+
only showing top 5 rows



lastException = null
defined class GenreAverage
averageRatingsDF = [genre: string, average_rating: double]


[genre: string, average_rating: double]